Activity 4

In [ ]:
# Shopping AI Assistant
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_vertexai import ChatVertexAI
from langgraph_swarm import create_swarm, create_handoff_tool


MODEL_NAME = "gemini-2.0-flash-001"
MODEL_KWARGS = {
    "temperature": 0.28,
    "max_output_tokens": 8192,
    "top_p": 0.95,
    "top_k": None,}

MODEL = ChatVertexAI(model_name=MODEL_NAME, **MODEL_KWARGS)

checkpointer = InMemorySaver()

def customer_service(query: str) -> str:
    """
    A tool that provides customer service responses.
    It returns a greeting message if the user greets, otherwise raises an exception.
    """
    greetings = ["hi", "hello", "good day", "hey"]
    if any(greet in query.lower() for greet in greetings):
        return "Thank you for reaching out! How can I assist you today?"
    # If not a greeting, raise Exception to trigger handoff
    raise Exception("Not a greeting.")

csr_agent = create_react_agent(
    model=MODEL,
    tools=[customer_service,create_handoff_tool(
            agent_name="Fashionista",
            description="Transfer to Fashionista for fashion-related queries (dress, jacket, shoes)."
        ),
        create_handoff_tool(
            agent_name="Gadgenista",
            description="Transfer to Gadgenista for gadget-related queries (laptop, smartphone, earbuds)."
        )
    ],
    name = "Customerista",
    prompt = """
    You are Customerista, a customer service representative. Greet the user if they say hi, hello, or similar. Otherwise, hand off to the right agent.
    """,
    checkpointer=checkpointer
)

def fashion_data(query: str) -> str:
    """
    A tool that provides product recommendations in the fashion category.
    It returns product names and prices based on the user's query.
    """
    catalog = {
        "dress": "Summer Floral Dress - $49",
        "jacket": "Denim Jacket - $89",
        "shoes": "Classic White Sneakers - $69"
    }
    for item in catalog:
        if item in query.lower():
            return catalog[item]
    raise Exception("Not a fashion query.")



fashion_agent = create_react_agent(
    model=MODEL,
    tools=[fashion_data, create_handoff_tool(
        agent_name="Gadgenista",
        description = "Transfer to Gadgenista, It can help you with gadgets such as a laptop, smartphone, and earbuds.",
        )
    ],
    name = "Fashionista",
    prompt = """
    You are Fashionista, fashion product recommender. Only use your tool to recommend fashion items.
    """,
    checkpointer=checkpointer
)

def gadget_data(query: str) -> str:
    """
    A tool that provides product recommendations in the gadgets category.
    It returns product names and prices based on the user's query.
    """
    catalog = {
        "laptop": "14-inch Ultrabook - $899",
        "smartphone": "Flagship Phone X - $999",
        "earbuds": "Noise-Canceling Earbuds - $199"
    }
    for item in catalog:
        if item in query.lower():
            return catalog[item]
    raise Exception("Not a gadget query.")

gadget_agent = create_react_agent(
    model=MODEL,
    tools=[gadget_data, create_handoff_tool(agent_name="Fashionista",
        description = "Transfer to Fashionista, It can help you with fashion such as a dress, jacket, and shoes.",
        )
    ],
    name = "Gadgenista",
    prompt = """
    You are Gadgenista, a gadget assistant. Recommend only from your tool.
    """,
    checkpointer=checkpointer
)

#swarms
swarms = create_swarm(
    [csr_agent, fashion_agent, gadget_agent],
    default_active_agent="Customerista",
).compile(checkpointer=checkpointer)

#supervisor
boss = create_supervisor(
    model=MODEL,
    agents =[fashion_agent, gadget_agent],
    prompt = """
    You are a supervisor agent. You interact with the user directly and help the user for queries regarding fashion and gadgets.

    For fashion related things, you'll consult Fashionista.
    For gadget related things, you'll consult Gadgenista.

    Only you and the user can interact, the user can't see the messages of the other agents, so relay the answer of the agents to the user.
    """,
    supervisor_name = "Bossing"
).compile(checkpointer=checkpointer)



In [4]:
#supervisor
response = boss.invoke(
    {"messages":[{"role": "user", "content": "Wassup!"}]},
    config={"configurable": {"thread_id": "1"}},
)

print(response['messages'][-1].content)

How can I help you today? Do you have any fashion or gadget-related questions?



In [9]:
#swarms
config = {"configurable": {"thread_id": "1"}}
response = swarms.invoke(
    {"messages":[{"role": "user", "content": "Hello! Can you recommend a smartphone"}]},
    config,
)

print(response['messages'][-1].name)
print('----------------------------------------')
print(response['messages'][-1].content)

Gadgenista
----------------------------------------
I recommend Flagship Phone X for $999.

